## Homework 7
- Goal: Train a deep learning neural network to classify insects images.
- Basic model structure
    - 3x3 convolution -> 3x3 convolution -> 2x2 max pool
    - 3x3 convolution -> 3x3 convolution -> 2x2 max pool
    - fully connected layer: 256 units
    - fully connected layer: 3 units (prediction class)
- Outcome
    - The accuracy on the test dataset is 87.2%.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

In [2]:
transform = transforms.Compose([transforms.Resize((320,384)),
                                transforms.ToTensor()])

In [3]:
?transforms.Resize

In [4]:
insect_train = datasets.ImageFolder('E:/Graduate_year_2/BIOS823/BIOS823HW/insects/insects/train/', transform=transform)

In [5]:
print('Number of samples: ', len(insect_train))

Number of samples:  1019


In [15]:
img, target = insect_train[13]
print('image size: ', img.size())

image size:  torch.Size([3, 320, 384])


In [7]:
insect_test = datasets.ImageFolder('E:/Graduate_year_2/BIOS823/BIOS823HW/insects/insects/test/', transform=transform)

In [8]:
print('Number of samples: ', len(insect_test))

Number of samples:  180


In [9]:
train_loader = torch.utils.data.DataLoader(insect_train, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(insect_test, batch_size =16, shuffle=True)

In [10]:
class INSECT_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(80*96*64, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        # conv layer 1
        x = self.conv1(x)
        x = F.relu(x)
        
        # conv layer 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        # conv layer 3
        x = self.conv3(x)
        x = F.relu(x)
        
        # conv layer 4
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        # fc layer 1
        x = x.view(-1, 80*96*64)
        x = self.fc1(x)
        x = F.relu(x)
        
        # fc layer 2
        x = self.fc2(x)
        return x    

In [11]:
model = INSECT_CNN()

In [16]:
print(model)

INSECT_CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=491520, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=3, bias=True)
)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Iterate through train set minibatchs 
for epoch in trange(3):  # <---- change here
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images  # <---- change here 
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

In [14]:
## Testing
correct = 0
total = len(insect_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))


Test accuracy: 0.8722222447395325


In [17]:
print('Test accuracy: {}'.format(correct/total))

Test accuracy: 0.8722222447395325
